In [ ]:
import os, inspect, sys
import datetime, json, gspread
import cPickle as pickle
from oauth2client.client import SignedJwtAssertionCredentials
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 16,8

In [ ]:
script_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.append(os.path.join(script_dir, os.pardir, os.pardir))

In [ ]:
from opengrid.library import config
c = config.Config()

In [ ]:
sys.path.append(c.get('tmpo', 'folder'))
import tmpo

try:
    if os.path.exists(c.get('tmpo', 'data')):
        path_to_tmpo_data = c.get('tmpo', 'data')
except:
    path_to_tmpo_data = None

# Class definitions

## Houseprint Singleton

The Houseprint is a Singleton object which contains all metadata for sites, devices and sensors. It can be pickled, saved and passed around

In [ ]:
class Houseprint(object):
    def __init__(self, spreadsheet = "Opengrid houseprint (Responses)"):
        """
            The Init method connects to the Google spreadsheet and parses the data into the OO structure.
        """
        
        self.sites = []
        self.parse_sheet(spreadsheet)
        self.timestamp = datetime.datetime.utcnow() #Add a timestamp upon creation
        
    def __repr__(self):
        return """
    Houseprint
    Created on {} (UTC)
    {} sites
    {} devices
    {} sensors
    """.format(self.timestamp,
               len(self.sites),
               sum([len(site.devices) for site in self.sites]),
               sum([len(site.sensors) for site in self.sites])
              )
    
    def parse_sheet(self, spreadsheet):

        print 'Opening connection to Houseprint sheet'
        #fetch credentials
        gjson = c.get('houseprint','json')
        json_key = json.load(open(gjson))
        scope = ['https://spreadsheets.google.com/feeds']
        credentials = SignedJwtAssertionCredentials(json_key['client_email'], json_key['private_key'], scope)
        
        #authorize and login
        gc = gspread.authorize(credentials)
        gc.login()
        
        #open sheets
        sheet = gc.open(spreadsheet)
        sites_sheet = sheet.worksheet('Accounts')
        devices_sheet = sheet.worksheet('Devices')
        sensors_sheet = sheet.worksheet('Sensors')
        
        self.parse_sites(sites_sheet)
        self.parse_devices(devices_sheet)
        self.parse_sensors(sensors_sheet)
        
    def parse_sites(self,sheet):
        
        records = sheet.get_all_records()
        
        for r in records:
            if r['Key'] == '': continue
            new_site = Site(hp = self, key = r['Key'])
            self.sites.append(new_site)
            
        print '{} Sites created'.format(len(self.sites))
        
    def parse_devices(self,sheet):
        
        records = sheet.get_all_records()
                        
        for r in records:
            if r['Key'] == '': continue  
            
            #find parent site and check if it exists
            site = self.find_site(r['Parent site'])
            if site is None:
                raise ValueError('Device {} was given an invalid site key {}'.format(r['Key'], r['Parent site']))
                
            #create a new device according to its manufacturer    
            if r['manufacturer'] == 'Flukso':
                new_device = Fluksometer(site = site, key = r['Key'])
            else:
                raise NotImplementedError('Devices from {} are not supported'.format(r['manufacturer']))
                
            #add new device to parent site
            site.devices.append(new_device)
            
        print '{} Devices created'.format(sum([len(site.devices) for site in self.sites]))
        
    def parse_sensors(self,sheet):
        
        records = sheet.get_all_records()
        
        for r in records:
            if r['Sensor_id'] == '': continue
            
            #find parent. If a parent device is specified, us that, otherwise use a parent site directly
            if r['parent device'] != '':
                device = self.find_device(r['parent device'])
                if device is None:
                    raise ValueError('Sensor {} was given an invalid device key {}. Leave the device field empty if you want to add a sensor without a device'.format(r['Sensor_id'], r['parent device']))
            else:
                site = self.find_site(r['parent site'])
                if site is None:
                    raise ValueError('Sensor {} was given an invalid site key {}'.format(r['Sensor_id'],r['parent site']))

            #create new sensor according to its manufacturer
            if r['manufacturer'] == 'Flukso':
                new_sensor = Fluksosensor(device = device,
                                         key = r['Sensor_id'],
                                         token = r['token'],
                                         type = r['sensor type'],
                                         description = r['name by user'],
                                         system = r['system'],
                                         quantity = r['quantity'],
                                         unit = r['unit'],
                                         direction = r['direction'],
                                         tariff = r['tariff'])
            else:
                raise NotImplementedError('Sensors from {} are not supported'.format(r['manufacturer']))

            #add sensor to device AND site
            if new_sensor.device is not None:
                new_sensor.device.sensors.append(new_sensor)
            new_sensor.site.sensors.append(new_sensor)
        
        print '{} sensors created'.format(sum([len(site.sensors) for site in self.sites]))
        
    def get_sensors(self, sensortype = None):
        """
            Return a list with all sensors
            
            Parameters
            ----------
            sensortype: gas, water, electricity: optional
        """
        res = []
        for site in self.sites:
            for sensor in site.get_sensors(sensortype = sensortype):
                res.append(sensor)
        return res
        
    def get_devices(self):
        """
            Return a list with all devices
        """
        res = []
        for site in self.sites:
            for device in site.devices:
                res.append(device)
        return res
    
    def find_site(self, key):
        '''
            Parameters
            ----------
            key: string
            
            Returns
            -------
            Site
        '''
        for site in self.sites:
            if site.key == key:
                return site
        return None
    
    def find_device(self, key):
        '''
            Parameters
            ----------
            key: string
            
            Returns
            -------
            Device
        '''
        for device in self.get_devices():
            if device.key == key:
                return device
        return None
    
    def find_sensor(self, key):
        '''
            Parameters
            ----------
            key: string
            
            Returns
            -------
            Sensor
        '''
        for sensor in self.get_sensors():
            if sensor.key == key:
                return sensor
        return None
    
    def save(self, filename):
        """
        Pickle the houseprint object
        
        Parameters
        ----------
        * filename : str
            Filename, if relative path or just filename, it is appended to the
            current working directory
        
        """
        if hasattr(self,'_tmpos'):
            #temporarily delete tmpo session
            tmpos_tmp = self._tmpos
            self._tmpos = None

        abspath = os.path.join(os.getcwd(), filename)
        f=file(abspath, 'w')
        pickle.dump(self, f)
        f.close()
        
        print("Saved houseprint to {}".format(abspath))
        
        if hasattr(self,'_tmpos'):
            #restore tmpo session
            self._tmpos = tmpos_tmp
        
    def init_tmpo(self, tmpos=None):
        """
            Fluksosensors need a tmpo session to obtain data.
            It is overkill to have each fluksosensor make its own session, syncing would take too long and be overly redundant.
            Passing a tmpo session to the get_data function is also bad form because we might add new types of sensors that don't use tmpo in the future.
            This is why the session is initialised here.
            
            A tmpo session as parameter is optional.
            If no session is passed, a new one will be created using the location in the config file.
        """
        
        if tmpos is not None:
            self._tmpos = tmpos
        else:
            self._tmpos = tmpo.Session(path_to_tmpo_data)
            
    def get_tmpos(self):
        """
            Returns
            -------
            TMPO session
        """
        if hasattr(self,'_tmpos'):
            return self._tmpos
        else:
            raise RuntimeError('No TMPO session was set, use the init_tmpo method to add or create a TMPO Session')
            
    def sync_tmpos(self):
        """
            Add all Fluksosensors to the TMPO session and sync
        """
        tmpos = self.get_tmpos()
        fluksosensors = [sensor for sensor in self.get_sensors() if isinstance(sensor,Fluksosensor)]
        
        for sensor in fluksosensors:
            tmpos.add(sensor.fluksoid, sensor.token)
            
        tmpos.sync()
        
    def get_data(self, sensortype = None, head = None, tail = None):
        """
            Return a Pandas Dataframe with joined data for all sensors in the houseprint
            
            Parameters
            ----------
            sensortype: gas, water, electricity: optional
            head, tail: timestamps
        """
        sensors = self.get_sensors(sensortype)
        series = [sensor.get_data(head=head,tail=tail) for sensor in sensors]
        return pd.concat(series, axis=1)

## Site

A Site is a physical entity (a house, appartment, school, or other building). It may contain multiple devices and sensors.

The Site contains most of the metadata, eg. the number of inhabitants, the size of the building, the location etc.

In [ ]:
class Site(object):
    def __init__(self, hp, key, postcode = None, size = None, inhabitants = None):
        self.hp = hp #backref to parent
        self.key = key
        self.postcode = postcode
        self.size = size
        self.inhabitants = inhabitants
        
        self.devices = []
        self.sensors = []
        
    def __repr__(self):
        return """
    Site
    Key: {}
    {} devices
    {} sensors
    """.format(self.key,
               len(self.devices),
               len(self.sensors)
              )
        
    def get_sensors(self, sensortype = None):
        """
            Return a list with all sensors
            
            Parameters
            ----------
            sensortype: gas, water, electricity: optional
        """
        return [sensor for sensor in self.sensors if sensor.type == sensortype or sensortype is None]
    
    def get_data(self, sensortype = None, head = None, tail = None):
        """
            Return a Pandas Dataframe with joined data for all sensors in this Site
            
            Parameters
            ----------
            sensortype: gas, water, electricity: optional
            head, tail: timestamps
        """
        sensors = self.get_sensors(sensortype)
        series = [sensor.get_data(head=head,tail=tail) for sensor in sensors]
        return pd.concat(series, axis=1)

## Device

A Device is an entity that can contain multiple sensors.
The generic Device class can be inherited by a specific device class, eg. Fluksometer

In [ ]:
class Device(object):
    def __init__(self, key, site):
        self.key = key
        self.site = site
        self.sensors = []
        
    def __repr__(self):
        return """
    {}
    Key: {}
    {} sensors
    """.format(self.__class__.__name__,
               self.key,
               len(self.sensors)
              )
        
    def get_sensors(self, sensortype = None):
        """
            Return a list with all sensors
            
            Parameters
            ----------
            sensortype: gas, water, electricity: optional
        """
        return [sensor for sensor in self.sensors if sensor.type == sensortype or sensortype is None]
    
    def get_data(self, sensortype = None, head = None, tail = None):
        """
            Return a Pandas Dataframe with the joined data for all sensors in this device
            
            Parameters
            ----------
            sensortype: gas, water, electricity: optional
            head, tail: timestamps indicating start and end
        """
        
        sensors = self.get_sensors(sensortype)
        series = [sensor.get_data(head=head,tail=tail) for sensor in sensors]
        return pd.concat(series, axis=1)

In [ ]:
class Fluksometer(Device):
    def __init__(self, site, key, mastertoken = None):
        
        #invoke init method of generic Device
        super(Fluksometer, self).__init__(key, site)
        
        self.mastertoken = mastertoken

## Sensor

A sensor generates a single data stream. It can have a parent device, but the possibility is also left open for a sensor to stand alone in a site.
It is an abstract class definition which has to be overridden (by eg. a Fluksosensor).

This class contains all metadata concerning the function and type of the sensor (eg. electricity - solar, ...)

In [ ]:
class Sensor(object):
    def __init__(self, key, device, site, type, description, system, quantity, unit, direction, tariff):
        self.key = key
        self.device = device
        self.site = site
        self.type = type
        self.description = description
        self.system = system
        self.quantity = quantity
        self.unit = unit
        self.direction = direction
        self.tariff = tariff
        
    def __repr__(self):
        return """
    {}
    Key: {}
    Type: {}
    """.format(self.__class__.__name__,
               self.key,
               self.type
              )
        
    def get_data(self, head = None, tail = None):
        """
            Return a Pandas Series with measurement data
            
            Parameters
            ----------
            head, tail: timestamps for the begin and end of the interval
            
            Notes
            -----
            This is an abstract method, because each type of sensor has a different way of fetching the data.
        """
        
        raise NotImplementedError("Subclass must implement abstract method")

In [ ]:
class Fluksosensor(Sensor):
    def __init__(self, key, token, device, type, description, system, quantity, unit, direction, tariff):
        
        #invoke init method of abstract Sensor
        super(Fluksosensor, self).__init__(key = key,
                                           device = device,
                                           site = device.site,
                                           type = type,
                                           description = description,
                                           system = system,
                                           quantity = quantity,
                                          unit = unit,
                                          direction = direction,
                                          tariff = tariff)
        
        if token != '':
            self.token = token
        else:
            self.token = parent.mastertoken
        
    # @Override :-D    
    def get_data(self, head = None, tail = None):
        '''
            Connect to tmpo and fetch a data series
            
            Parameters
            ----------
            head, tail: optional timestamps
        '''

        tmpos = self.site.hp.get_tmpos()
        
        if head is None:
            head = 0
        if tail is None:
            tail = tmpo.EPOCHS_MAX
        
        data = tmpos.series(sid = self.key,
                           head = head,
                           tail = tail)
        
        return data

#Test

##Houseprint

In [ ]:
hp = Houseprint()

In [ ]:
hp

In [ ]:
hp.sites

In [ ]:
hp.get_devices()

In [ ]:
hp.get_sensors('water')

###TMPO

The houseprint, sites, devices and sensors all have a get_data method. In order to get these working for the fluksosensors, the houseprint needs a tmpo session

In [ ]:
hp.init_tmpo()

In [ ]:
#hp.sync_tmpos()

A Houseprint object can be saved as a pickle. It loses its tmpo session however (connections cannot be pickled)

In [ ]:
hp.save('new_houseprint.pkl')

###Lookup sites, devices, sensors

In [ ]:
hp.find_site(1)

In [ ]:
hp.find_device('FL03001552')

In [ ]:
hp.find_sensor('53b1eb0479c83dee927fff10b0cb0fe6')

###Get Data

In [ ]:
head = pd.Timestamp('20150617')
tail = pd.Timestamp('20150618')
hp.get_data(sensortype='electricity', head=head,tail=tail).plot()

##Site

In [ ]:
site = hp.find_site(1)
site

In [ ]:
site.key

In [ ]:
site.devices

In [ ]:
site.get_sensors('electricity')

In [ ]:
head = pd.Timestamp('20150617')
tail = pd.Timestamp('20150618')
site.get_data(sensortype='electricity', head=head,tail=tail).plot()

##Device

In [ ]:
device = hp.find_device('FL03001552')
device

In [ ]:
device.key

In [ ]:
device.get_sensors('gas')

In [ ]:
head = pd.Timestamp('20150617')
tail = pd.Timestamp('20150618')
device.get_data(sensortype='electricity', head=head,tail=tail).plot()

##Sensor

In [ ]:
sensor = hp.find_sensor('53b1eb0479c83dee927fff10b0cb0fe6')
sensor

In [ ]:
sensor.key

In [ ]:
sensor.type

In [ ]:
sensor.description

In [ ]:
sensor.system

In [ ]:
sensor.unit

In [ ]:
head = pd.Timestamp('20150617')
tail = pd.Timestamp('20150618')
sensor.get_data(head,tail).plot()